In [11]:
! pip install BeautifulSoup4

In [6]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
from urllib.request import Request

In [7]:
import pandas as pd
import numpy as np

In [1286]:
my_url = 'https://seekingalpha.com/earnings/earnings-call-transcripts/3'

In [1287]:
User_Agent = Request(my_url , headers = {'User-Agent' : 'Mozilla/5.0'})
uClient = uReq(User_Agent)



page_html = uClient.read() #offload data in a variable

   

uClient.close()   #close the client

### Get all the transcript links from the given page

In [1290]:
page_soup = soup(page_html , "html.parser")

### Extract each link

In [1292]:
import re
p=[]
ul = page_soup.find('ul' , {'class' : 'list-group sa-base-article-list'})
ul
for x in ul.findAll('a' , href=re.compile('^/article.*[^r]$')):
    p.append(x.get('href'))
    
p[0]

'/article/4393864-campbell-soup-company-cpb-ceo-mark-clouse-on-q1-2021-results-earnings-call-transcript'

### Modify these links as Clickable

In [1295]:
mystr = 'https://seekingalpha.com'

In [1296]:
p = [mystr + str(x) for x in p]

In [1298]:
for i in range(len(p)):
    print(p[i])

https://seekingalpha.com/article/4393864-campbell-soup-company-cpb-ceo-mark-clouse-on-q1-2021-results-earnings-call-transcript
https://seekingalpha.com/article/4393863-phreesia-inc-s-phr-ceo-chaim-indig-on-q3-2021-results-earnings-call-transcript
https://seekingalpha.com/article/4393858-archer-daniels-midland-company-adm-presents-bmo-growth-esg-conference-transcript
https://seekingalpha.com/article/4393857-crispr-therapeutics-ag-crsp-phase-1-2-ctx001-investor-update-transcript
https://seekingalpha.com/article/4393853-fireeye-inc-feye-ceo-kevin-mandia-presents-ubs-global-tmt-virtual-conference-transcript
https://seekingalpha.com/article/4393833-british-american-tobacco-p-l-c-bti-q4-2020-trading-update-conference-call-transcript
https://seekingalpha.com/article/4393831-designer-brands-inc-dbi-ceo-roger-rawlins-on-q3-2020-results-earnings-call-transcript
https://seekingalpha.com/article/4393823-chewy-inc-s-chwy-ceo-sumit-singh-on-q3-2020-results-earnings-call-transcript
https://seekingalp

### Selenium - loop through Links and extract data

In [233]:
! pip install selenium

In [892]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
import time

In [1336]:
driver = webdriver.Chrome(executable_path = "C:/Users/Hard Thakkar/Documents/chromedriver_win32/chromedriver.exe")

In [1218]:
paper = p[0]
paper

'https://seekingalpha.com/article/4393815-sumo-logic-inc-sumo-ceo-ramin-sayar-on-q3-2021-results-earnings-call-transcript'

# Encapsulating Everything and Appending the Final DataFrame

In [1303]:
for i in range(len(p)):
    try:
        wait_time_out = 20
        driver.get(p[i])
        page_soup1 = soup(driver.page_source, "html.parser")
        #article = page_soup1.find('div' , {'id' : 'a-body'})
        article_data = page_soup1.find("article")

    
        list_df = []
        for i in article_data.findAll('p') :
            list_df.append(i.get_text())

    
        date_loop = get_date(article_data)[0]
        time_loop = get_time(article_data)[0]
        Ticker = ticker(article_data)
        Company = company(article_data)
        Title = title(article_data)
        Org = get_Org(article_data)

    
    
        speech_list =[]
        for i in list_df:
            if len(i)>150:
                speech_list.append(list_df[list_df.index(i)-1])
                speech_list.append(i)
        speech_list = [speech_list]
    
    
    
    
        c_participant = []
        c_participant.extend(list_df[list_df.index('Company Participants')+1:list_df.index('Conference Call Participants')])
        c_participant
    
    
    
        participant_name=[]
        for i in c_participant:
            participant_name.extend([i.split('-',1)[0]])
        
        
        participant_type=[]
        for i in c_participant:
            participant_type.extend([i.split('-',1)[1]])
        
        
    
        for i in range(len(participant_name)):
            df_dict = {'Ticker' : Ticker ,
                    'Company' : Company , 
                    'Title' : title(article_data)[0] ,
                    'Date' : date_loop , 
                    'Time' : time_loop, 
                    'Speech' : speech_list , 
                    'Participant Name' : participant_name[i] , 
                    'Participant Type' : participant_type[i] , 
                    'Participant Organization' : Org
                    }
    
            final_df = final_df.append(df_dict , ignore_index=True)



        driver.back()
    except:
        pass

# THIS IS OUR FINAL STRUCTURED DATAFRAME FROM WEB SCRAPING

In [1361]:
final_df

,Ticker,Company,Title,Date,Time,Participant Name,Participant Type,Participant Organization
0,DBI,Designer Brands Inc.,Q3 2020 Results - Earnings Call Transcript,"Dec.9,2020",10:07AM,Stacy Turnof,"Investor Relations, Edelman",Designer Brands Inc.
1,DBI,Designer Brands Inc.,Q3 2020 Results - Earnings Call Transcript,"Dec.9,2020",10:07AM,Roger Rawlins,Chief Executive Officer,Designer Brands Inc.
2,DBI,Designer Brands Inc.,Q3 2020 Results - Earnings Call Transcript,"Dec.9,2020",10:07AM,Jared Poff,Chief Financial Officer,Designer Brands Inc.
3,CHWY,"Chewy, Inc.",Q3 2020 Results - Earnings Call Transcript,"Dec.9,2020",9:44,Robert LaFleur,Vice President of Investor Relations & Capita...,"Chewy, Inc."
4,CHWY,"Chewy, Inc.",Q3 2020 Results - Earnings Call Transcript,"Dec.9,2020",9:44,Sumit Singh,Chief Executive Officer,"Chewy, Inc."
5,CHWY,"Chewy, Inc.",Q3 2020 Results - Earnings Call Transcript,"Dec.9,2020",9:44,Mario Marte,Chief Financial Officer,"Chewy, Inc."
6,SUMO,"[Sumo Logic, Inc. ]",Q3 2021 Results - Earnings Call Transcript,"Dec.9,2020",9:10,Paul Thomas,IR,"Sumo Logic, Inc."
7,SUMO,"[Sumo Logic, Inc. ]",Q3 2021 Results - Earnings Call Transcript,"Dec.9,2020",9:10,Ramin Sayar,CEO,"Sumo Logic, Inc."
8,SUMO,"[Sumo Logic, Inc. ]",Q3 2021 Results - Earnings Call Transcript,"Dec.9,2020",9:10,Sydney Carey,CFO,"Sumo Logic, Inc."
9,GWRE,"[Guidewire Software, Inc.'s ]",Q1 2021 Results - Earnings Call Transcript,"Dec.9,2020",1:38,Alex Hughes,"Vice President, Investor Relations","Guidewire Software, Inc."


# Functions and Code for all columns needed

### Date And Time

In [1164]:
def get_date(article):
    date = article.find("time").get_text()
    #time = time.split(" " , 1)
    date = re.split(' ', date)
    date = [''.join(date[0:4])]
    
    return date

In [1165]:
def get_time(article):
    time = article.find("time").get_text()
    #time = time.split(" " , 1)
    time = re.split(' ', time)
    time = [''.join(time[4:6])]
    
    return time

In [1215]:
get_date(article_data)[0]

['Dec.9,2020']

In [1167]:
get_time(article_data)[0]

'9:44'

In [1325]:
list_df = []
for i in article_data.findAll('p') :
    list_df.append(i.get_text())
#x = article_data.findAll('p')
#for i in x:
#    list_df.append(i.findAll('p'))

In [1326]:
list_df

['Newmont Corporation (NYSE:NEM) 2021 Investor Update Conference Call December  8, 2020  9:00 AM ET',
 'Company Participants',
 'Eric Colby - Vice President of Investor Relations and Global Communications',
 'Tom Palmer - President and Chief Executive Officer',
 'Rob Atkinson - Chief Operating Officer',
 'Nancy Buese - Chief Financial Officer',
 'Randy Engel - Executive Vice President, Strategic Development',
 'Dean Gehring - Executive Vice President and Chief Technology Officer',
 'Stephen Gottesfeld - Executive Vice President and Chief Sustainability Officer',
 'Conference Call Participants',
 'Fahad Tariq - Credit Suisse',
 'Josh Wolfson - RBC Capital Markets',
 'Jackie Przybylowski - BMO Capital',
 'Anita Soni - CIBC World Markets',
 'Mike Jalonen - Bank of America',
 'Greg Barnes - TD Securities',
 'Tyler Langton - JPMorgan',
 'Brian MacArthur - Raymond James',
 'Michael Dudas - Vertical Research Partners',
 'John Tumazos - Very Independent Research',
 'Operator',
 'Good morning a

# Code for Conference Call participants

In [1327]:
cc_participant = []

cc_participant.extend(list_df[list_df.index('Conference Call Participants')+1:list_df.index('Operator')])

cc_participant

['Fahad Tariq - Credit Suisse',
 'Josh Wolfson - RBC Capital Markets',
 'Jackie Przybylowski - BMO Capital',
 'Anita Soni - CIBC World Markets',
 'Mike Jalonen - Bank of America',
 'Greg Barnes - TD Securities',
 'Tyler Langton - JPMorgan',
 'Brian MacArthur - Raymond James',
 'Michael Dudas - Vertical Research Partners',
 'John Tumazos - Very Independent Research']

In [1329]:
participant_name2=[]
for i in cc_participant:
    participant_name2.extend([i.split('-',1)[0]])
participant_name2

['Fahad Tariq ',
 'Josh Wolfson ',
 'Jackie Przybylowski ',
 'Anita Soni ',
 'Mike Jalonen ',
 'Greg Barnes ',
 'Tyler Langton ',
 'Brian MacArthur ',
 'Michael Dudas ',
 'John Tumazos ']

In [1332]:
participant_org=[]
for i in cc_participant:
    participant_org.extend([i.split('-',1)[1]])

### Ticker

In [1171]:
def ticker(article):
    ticker = article.find("h1").get_text()
    #ticker = ticker.get_text()
    #ticker = ticker.get_text()
    ticker = ticker[ticker.find('(')+1:ticker.find(')')]
    #ticker=[ticker]
    return ticker

In [1192]:
ticker(article_data)

'CHWY'

### Company

In [1346]:
def company(article):
    company = page_soup1.find("h1")
    company = company.get_text()
    company = company[:company.find('(')]
    #company=[company]
    return company

In [1347]:
company(article_data)

'Newmont Corporation '

### Title

In [1175]:
def title(article):
    title = article.find("h1")
    title = title.get_text()
    title = title[title.find('on')+3:]
    title=[title]
    return title

In [1176]:
title(article_data)

['Q3 2020 Results - Earnings Call Transcript']

### SPEECH/CONTENT

In [1177]:
speech_list =[]
for i in list_df:
    if len(i)>150:
        speech_list.append(list_df[list_df.index(i)-1])
        speech_list.append(i)
        

speech_list = [speech_list]

In [1178]:
speech_list

[['Operator',
  'Good day, and welcome to the Chewy Third Quarter 2020 Earnings Conference Call. All participants will be in a listen-only mode. [Operator Instructions] After today’s presentation, there will be an opportunity to ask questions. [Operator Instructions] Please note that this event is being recorded.',
  'Robert LaFleur',
  "Thank you for joining us on the call today to discuss our third quarter fiscal 2020 results. Joining me today are Chewy's CEO, Sumit Singh; and CFO, Mario Marte. Our earnings release and letter to shareholders, which were filed with the SEC on Form 8-K earlier today, have been posted to the Investor Relations section of our website investor.chewy.com. A link to the webcast of today's conference call is also available on our site.",
  "Thank you for joining us on the call today to discuss our third quarter fiscal 2020 results. Joining me today are Chewy's CEO, Sumit Singh; and CFO, Mario Marte. Our earnings release and letter to shareholders, which were

### Participant info

In [1180]:

c_participant = []

c_participant.extend(list_df[list_df.index('Company Participants')+1:list_df.index('Conference Call Participants')])

c_participant

['Robert LaFleur - Vice President of Investor Relations & Capital Markets',
 'Sumit Singh - Chief Executive Officer',
 'Mario Marte - Chief Financial Officer']

### Participant Name

In [1181]:
participant_name=[]
for i in c_participant:
    participant_name.extend([i.split('-',1)[0]])
participant_name

['Robert LaFleur ', 'Sumit Singh ', 'Mario Marte ']

### Participant Type

In [1182]:
participant_type=[]
for i in c_participant:
    participant_type.extend([i.split('-',1)[1]])


In [1183]:
participant_type

[' Vice President of Investor Relations & Capital Markets',
 ' Chief Executive Officer',
 ' Chief Financial Officer']

### Participation Organization

In [1184]:
def get_Org(article):
    org = article.find('p').get_text()
    org = org.split("  " , 1)
    org = org[0]
    org = org[0:org.index('(')]
    return org

In [1185]:
get_Org(article_data)

'Chewy, Inc. '

# Code for Final Dataframe (Also used in the main scraping function)

In [1151]:
final_df = pd.DataFrame(columns=['Ticker', 'Company', 'Title' , 'Date' , 'Time' , 'Speech' , 'Participant Name' , 'Participant Type' , 'Participant Organization'])

In [1152]:
final_df

,Ticker,Company,Title,Date,Time,Speech,Participant Name,Participant Type,Participant Organization


In [1186]:
for i in range(len(participant_name)):
    df_dict = {'Ticker' : ticker(article_data) , 'Company' : company(article_data)[0] , 'Title' : title(article_data)[0] , 'Date' : get_date(article_data)[0] , 'Time' : get_time(article_data)[0] , 'Speech' : speech_list , 'Participant Name' : participant_name[i] , 'Participant Type' : participant_type[i] , 'Participant Organization' : get_Org(article_data)}
    final_df = final_df.append(df_dict , ignore_index=True)

In [1187]:
final_df

,Ticker,Company,Title,Date,Time,Speech,Participant Name,Participant Type,Participant Organization
0,DBI,Designer Brands Inc.,Q3 2020 Results - Earnings Call Transcript,"Dec.9,2020",10:07AM,"[[Operator, Good day and welcome to the Design...",Stacy Turnof,"Investor Relations, Edelman",Designer Brands Inc.
1,DBI,Designer Brands Inc.,Q3 2020 Results - Earnings Call Transcript,"Dec.9,2020",10:07AM,"[[Operator, Good day and welcome to the Design...",Roger Rawlins,Chief Executive Officer,Designer Brands Inc.
2,DBI,Designer Brands Inc.,Q3 2020 Results - Earnings Call Transcript,"Dec.9,2020",10:07AM,"[[Operator, Good day and welcome to the Design...",Jared Poff,Chief Financial Officer,Designer Brands Inc.
3,CHWY,"Chewy, Inc.",Q3 2020 Results - Earnings Call Transcript,"Dec.9,2020",9:44,"[[Operator, Good day, and welcome to the Chewy...",Robert LaFleur,Vice President of Investor Relations & Capita...,"Chewy, Inc."
4,CHWY,"Chewy, Inc.",Q3 2020 Results - Earnings Call Transcript,"Dec.9,2020",9:44,"[[Operator, Good day, and welcome to the Chewy...",Sumit Singh,Chief Executive Officer,"Chewy, Inc."
5,CHWY,"Chewy, Inc.",Q3 2020 Results - Earnings Call Transcript,"Dec.9,2020",9:44,"[[Operator, Good day, and welcome to the Chewy...",Mario Marte,Chief Financial Officer,"Chewy, Inc."


In [351]:
#for i in paper:
#    wait_time_out = 15
#    driver.get(i)
#    wait_variable = WebDriverWait(driver , 15)
#    l = wait_variable.until(EC.element_to_be_clickable((By.NAME, links)))
    

In [265]:
#wait_time_out = 10
#for links in i:
#    get_url = driver.get(links)
#    get_url.click()
#    try :
#        element = WebDriverWait(driver , 10).until(EC.element_to_be_clickable((By.NAME, links)))
#    finally:
#        driver.quit()
        
#    break

    
    


AttributeError: 'NoneType' object has no attribute 'click'

In [ ]:
#useless but helpful

#final_df['Participant Name'] = c_participant

#final_df[['Participant Name' , 'Participant Type']] = final_df['Participant Name'].str.split('-' ,1,expand=True)

#, df , column
# df[column] = np.array(struct).flatten()

#participant = []
#Participant_info(list_df , participant)
#participant


#final_df['dummy' , 'dummy2'] = final_df['Participant'].str.split('-' , 1)

#final_df['dummy'].split



In [ ]:
#cc_participant.extend(list_df[list_df.index('Conference Call Participants')+1:list_df.index('Operator')])

#for i in c_participant:
#    c_participant.append(str(i).split('-',1))

#dummy_df = pd.DataFrame()
#dummy_df[["hello" , "h"]] = dummy2
#dummy2
#for x in cc_participant:
#    c_participant.append(x)
#c_participant = c_participant + cc_participant
#c_participant



#d_df = pd.DataFrame(c_participant , columns = ['hello'])
#d_df[['First','Last']] = d_df.hello.str.split('-' ,1,expand=True)
#d_df
#c_participant = c_participant+cc_participant